In [50]:
import numpy as np
from osgeo import gdal, ogr
from sklearn import metrics
import pandas as pd

naip_fn = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/verschiedene Stacks/Stack_rgb_dem_ndvi_nir.tif"

driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)

test_fn = "D:/cbr/VP2/VP2_SoilMapping/Shapes/test_klassiert.shp"
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)

truth = target_ds.GetRasterBand(1).ReadAsArray()
pred_ds = gdal.Open('D:/cbr/VP2/VP2_SoilMapping/Processed/classified_Wetzwil_mitvierKlassen.tif')
pred = pred_ds.GetRasterBand(1).ReadAsArray()

idx = np.nonzero(truth)

cm = metrics.confusion_matrix(truth[idx], pred[idx])
kappa = metrics.cohen_kappa_score(truth[idx], pred[idx])
precision = metrics.precision_score(truth[idx], pred[idx], average='weighted')
recall = metrics.recall_score(truth[idx], pred[idx], average='weighted')
accuracy = metrics.accuracy_score(truth[idx], pred[idx])
f1 = metrics.f1_score(truth[idx], pred[idx], average='weighted')
#roc = metrics.roc_curve(truth[idx], pred[idx])
#k_accuracy = metrics.top_k_accuracy_score(truth[idx], pred[idx])

corr_matrix = pd.DataFrame(data=cm, index=["Grass", "Other", "Grey Cluster", "Possible Grey"], columns=["Pred. Grass", "Pred. Other", "Pred. Grey Cluster", "Pred. Possible Grey"])
print(corr_matrix)

print(" ")
print("######## Statistics ######### ")

#accuracy = cm.diagonal() / cm.sum(axis=0)
#print("Accuracy :                   ", accuracy)

print(cm)
print("Kappa :", kappa)
print("Precision :", precision)
print("Recall :", recall)
print("Accuracy :", accuracy)
print("F1 :", f1)
#print("Kappa :", roc)
#print("Kappa :", k_accuracy)

#A = cm[0][0]
#B = cm[0][1]
#C = cm[1][0]
#D = cm[1][1]
#P_corr = ((A + B) / (A + B + C + D)) * ((A + C) / (A + B + C + D))
#P_incorr = ((C + D) / (A + B + C + D)) * ((B + D) / ( A + B + C + D))
#Kappa = P_corr + P_incorr
#print("Kappa Index :                ", Kappa)
#
#FAR = (cm[0][1]) / ((cm[0][1]) + (cm[0][0]))
#print("False Alarm Ratio :          ", FAR)
#
#POD = (cm[0][0]) / ((cm[0][0]) + (cm[1][0]))
#print("Propability of Detection :    ",POD)


               Pred. Grass  Pred. Other  Pred. Grey Cluster  \
Grass                    5            2                   0   
Other                    0           16                   0   
Grey Cluster             0            2                   6   
Possible Grey            0            4                   0   

               Pred. Possible Grey  
Grass                            0  
Other                            0  
Grey Cluster                     0  
Possible Grey                    0  
 
######## Statistics ######### 
[[ 5  2  0  0]
 [ 0 16  0  0]
 [ 0  2  6  0]
 [ 0  4  0  0]]
Kappa : 0.6306068601583114
Precision : 0.7333333333333333
Recall : 0.7714285714285715
Accuracy : 0.7714285714285715


C:\Users\st1154414\.conda\envs\pygdal\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
